In [ ]:
# 載入必要套件
from BackTest import Performance
import pandas as pd
import os
import yfinance as yf
import requests
import sys

datapath = './'

'''自訂函數'''


# 取得資料：本地或是 Yahoo
def getDataFM(prod, st, en):
    # 檔案路徑
    bakfile = f"data/YF_{prod}_{st}_{en}_stock_daily_adj.csv"
    # 假如已經有 Excel 資料檔案
    if os.path.exists(bakfile):
        # 讀取 Excel 資料檔案
        data = pd.read_csv(bakfile, index_col="Date", parse_dates=True)
    # 假如 Excel 資料檔案不存在
    else:
        # 透過 yf 下載
        data = yf.download(f"{prod}.TW", start=st, end=en)
        # 下載的資料是英文的，處理欄位名稱，轉換為小寫
        # 轉換部分不包含索引，所以日期還是大寫開頭的 Date
        data.columns = [i.lower() for i in data.columns]
        # 假如下載後沒有資料
        if data.empty:
            print("無法從 Yahoo Finance 下載資料")
            # 傳回一個空的 pd
            return pd.DataFrame()
        # 假如資料 data 不是空的，依照指定路徑存擋
        data.to_csv(bakfile)
    # 存檔後，傳回資料
    return data


# (1)取得所有股票商品代碼
def getStockList():
    # 檔案名稱
    bakfile = f"{datapath}/TSE_StockList.csv"
    # 假如有檔案
    if os.path.exists(bakfile):
        # 取得檔案數據
        df = pd.read_csv(bakfile)
    # 沒有的話就取檔案內容
    else:
        # 取得檔案內容
        res = requests.get("http://isin.twse.com.tw/isin/C_public.jsp?strMode=2")
        df = pd.read_html(res.text)[0]
        df.columns = df.iloc[0]
        df = df.iloc[2:]
        df = df.dropna(thresh=3, axis=0).dropna(thresh=3, axis=1)
        df.to_csv(bakfile)
    # 回傳資料
    return df


if __name__ == "__main__":
    # 取得股票代碼
    stock_list = getStockList()
    # 取得半導體業的股票代碼
    stocks = (
        stock_list[(stock_list["產業別"] == "半導體業") & (stock_list["市場別"] == "上市")]["有價證券代號及名稱"]
        .str.split("　")
        .str[0]
    )

    # 將每個同產業的股票依序回測
    for prod in stocks:
        print(prod)
        # 取得回測資料
        try:
            data = getDataFM(prod, "2022-01-01", "2022-12-31")
        except Exception as e:
            print(e)
            continue

        # 計算簡單移動平均線
        data["ma1"] = data["close"].rolling(60).mean()
        data["ma2"] = data["close"].rolling(120).mean()

        # 初始部位
        position = 0
        trade = pd.DataFrame()
        # 開始回測
        for i in range(data.shape[0] - 1):
            # 取得策略會應用到的變數
            c_time = data.index[i]
            c_high = data.loc[c_time, "high"]
            c_close = data.loc[c_time, "close"]
            c_ma1 = data.loc[c_time, "ma1"]
            c_ma2 = data.loc[c_time, "ma2"]
            # 取下一期資料做為進場資料
            n_time = data.index[i + 1]
            n_open = data.loc[n_time, "open"]

            # 進場程序
            if position == 0:
                # 進場邏輯
                if c_ma1 > c_ma2:
                    position = 1
                    order_i = i
                    order_time = n_time
                    order_price = n_open
                    order_unit = 1
            # 出場程序
            elif position == 1:
                # 出場邏輯
                if c_ma1 < c_ma2:
                    position = 0
                    cover_time = n_time
                    cover_price = n_open
                    # 交易紀錄
                    trade = trade._append(
                        pd.Series(
                            [
                                prod,
                                "Buy",
                                order_time,
                                order_price,
                                cover_time,
                                cover_price,
                                order_unit,
                            ]
                        ),
                        ignore_index=True,
                    )

        print("回測商品代碼:", prod)
        Performance(trade, "Stock")
        print("----------------------------------")
